## Importing Libraries

In [22]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

## Set the MLflow Tracking URI

In [23]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

## Load training data and train a simple model

In [24]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

## Define an MLflow Experiment

In [25]:
mlflow.set_experiment("MLflowDemo")

<Experiment: artifact_location='/Users/gingermale/Documents/repos/mlflow/mlruns/4', creation_time=1728654601736, experiment_id='4', last_update_time=1728654601736, lifecycle_stage='active', name='MLflowDemo', tags={}>

## Log the model, hyperparameters, and loss metrics to MLflow.

In [26]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-demo",
    )

Registered model 'tracking-demo' already exists. Creating a new version of this model...
2024/10/24 13:41:22 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-demo, version 11
Created version '11' of model 'tracking-demo'.
2024/10/24 13:41:22 INFO mlflow.tracking._tracking_service.client: 🏃 View run loud-kit-737 at: http://127.0.0.1:8080/#/experiments/4/runs/3e8896e8d62e4cbbbb4af1d7a460039d.
2024/10/24 13:41:22 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/4.


## Load our saved model as a Python Function

In [27]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [28]:
print(model_info.model_uri)

runs:/3e8896e8d62e4cbbbb4af1d7a460039d/iris_model


## Use our model to predict the iris class type on a Pandas DataFrame

In [52]:
predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
